# Trying Ensembling

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import xgboost as xgb

from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('../data/Final Data/pct-diff-mlb-games-with-preds.csv')

In [3]:
df.head()

,home_team,away_team,Y,M,avg_pct_diff,obp_pct_diff,slg_pct_diff,team_ERA_pct_diff,team_WHIP_pct_diff,team_W-L_pct_diff,...,knn_pred,nn_pred,xgb_proba,knn_proba,nn_proba,home_win,knn_no_ops_pred,knn_no_ops_proba,knn_no_batting_pred,knn_no_batting_proba
0,KCA,NYN,2016,4,8.620726,3.092675,2.578581,7.754011,8.007859,5.263158,...,1,1,0.562908,0.533333,0.590414,1,1,0.533333,1,0.566667
1,PIT,SLN,2016,4,2.309712,0.376945,-0.040732,8.978328,-1.211283,-2.040816,...,0,1,0.533676,0.493333,0.563689,1,1,0.506667,1,0.513333
2,TBA,TOR,2016,4,-5.864577,-7.555239,-12.023378,-1.871658,1.468752,-16.250000,...,1,1,0.454400,0.566667,0.666198,0,1,0.593333,1,0.613333
3,ANA,CHN,2016,4,0.398896,-4.950458,-0.794957,14.720812,8.900281,-14.117647,...,0,1,0.450952,0.460000,0.531114,0,0,0.420000,0,0.500000
4,ARI,COL,2016,4,-1.095675,2.427164,-5.196744,-24.444444,-13.778463,13.924051,...,0,1,0.584152,0.446667,0.562535,0,0,0.466667,0,0.500000


In [4]:
nn_pred = df.nn_pred
knn_no_batting_pred = df.knn_no_batting_pred

y_test = df.home_win

### Max Accuracy

Below, when only using 'nn_pred' and 'knn_no_batting_pred' predictions the max accuracy we can get is 78.5%. 

In [5]:
num_correct = 0

for i in range(y_test.shape[0]):
    if y_test.values[i] in [nn_pred[i], knn_no_batting_pred[i]]:
        num_correct += 1

print(f'Maximum possible accuracy = {100*num_correct / y_test.shape[0]:.2f}%')

Maximum possible accuracy = 78.56%


Now, we need to figure out how to increase closer to that accuracy using those two sets of predictions. Below, we will make two new columns identifying if that row saw predictions that agreed (1,1 or 0,0) or if it has predictions that disagree (0,1 or 1,0).

In [6]:
conditions = [(df['knn_no_batting_pred'] != df['nn_pred'])]     #Sets the conditions
choices = [1]
df['mixed_nn_knn'] = np.select(conditions, choices, 0)

In [7]:
conditions = [(df['knn_no_batting_pred'] == df['nn_pred'])]     #Sets the conditions
choices = [1]
df['agree_nn_knn'] = np.select(conditions, choices, 0)

Now, we can make seperate dataframes to analyze these.

In [8]:
disagree_df = df[df['mixed_nn_knn'] == 1]
agree_df = df[df['agree_nn_knn'] == 1]

First, we know that if the two models agree then no matter what we do that will be the final prediction of the ensemble. Therefore right off the bat we are going to take that hit of getting that many wrong and that many right. The percent below is a representation of of the rows that agree how many does the models get right. 

In [12]:
agree_correct_pct = agree_df[agree_df['nn_pred'] == agree_df['home_win']].shape[0] / agree_df.shape[0]

print(f'{round(agree_correct_pct*100,2)}%')

64.61%


Now we can find the number 3805 which is the actual number correct so far since we know that if the models agree the ensemble has to make that prediction. Since we are just playing a numbers game, we also know that if we want to have 70% accuracy we need to have 6,803 correct predictions (9718 * 0.7).

In [13]:
round(agree_df.shape[0]*agree_correct_pct)

3805

Thus now we can figure out that of the ones that disagree (3,829) we need to get 2,998 correct. This is a percentage of 78% accuracy. Now the question is how do we do that.

In [15]:
round((9718*.7)-round(agree_df.shape[0]*agree_correct_pct))

2998

In [19]:
round((9718*.7)-round(agree_df.shape[0]*agree_correct_pct)) / disagree_df.shape[0]

0.7829720553669366

I have a different notebook with scratch work trying to figure this out. One issue is that using just the predictions doesn't work even if I try to use XGB as well. We are going to have to find a new way to determine this. One thought I had was to take the disagree ones and rerun them through a new model that hopefully will perform better. (I think that technique is called a meta-model {Not completely sure}). I am looking into the characteristics of the disagree ones as well to try to find an anomaly within the data.